tgb - 6/12/2021 - The goal is to see whether it would be possible to train a NN/MLR outputting results in quantile space while still penalizing them following the mean squared error in physical space.

tgb - 4/15/2021 - Recycling this notebook but fitting in percentile space (no scale_dict, use output in percentile units)

tgb - 4/15/2020
- Adapting Ankitesh's notebook that builds and train a "brute-force" network to David Walling's hyperparameter search  
- Adding the option to choose between aquaplanet and real-geography data

In [1]:
import sys
sys.path.insert(1,"/home1/07064/tg863631/anaconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages") #work around for h5py
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
from cbrain.climate_invariant import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
# from climate_invariant import *
from tensorflow.keras import layers
import datetime
from climate_invariant_utils import *
import yaml


/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


## Global Variables

In [2]:
# Load coordinates (just pick any file from the climate model run)

# Comet path below
# coor = xr.open_dataset("/oasis/scratch/comet/ankitesh/temp_project/data/sp8fbp_minus4k.cam2.h1.0000-01-01-00000.nc",\
#                     decode_times=False)

# GP path below
path_0K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/'
coor = xr.open_dataset(path_0K+"AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-09-02-00000.nc")

lat = coor.lat; lon = coor.lon; lev = coor.lev;
coor.close();

In [3]:
# Comet path below
# TRAINDIR = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/CRHData/'
# path = '/home/ankitesh/CBrain_project/CBRAIN-CAM/cbrain/'

# GP path below
TRAINDIR = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path = '/export/nfs0home/tbeucler/CBRAIN-CAM/cbrain/'
path_nnconfig = '/export/nfs0home/tbeucler/CBRAIN-CAM/nn_config/'

# Load hyam and hybm to calculate pressure field in SPCAM
path_hyam = 'hyam_hybm.pkl'
hf = open(path+path_hyam,'rb')
hyam,hybm = pickle.load(hf)

# Scale dictionary to convert the loss to W/m2
scale_dict = load_pickle(path_nnconfig+'scale_dicts/009_Wm2_scaling.pkl')

New Data generator class for the climate-invariant network. Calculates the physical rescalings needed to make the NN climate-invariant

## Data Generators

### Choose between aquaplanet and realistic geography here

In [4]:
# GP paths below
#path_aquaplanet = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
#path_realgeography = ''

# GP /fast paths below
path_aquaplanet = '/fast/tbeucler/climate_invariant/aquaplanet/'

# Comet paths below
# path_aquaplanet = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'
# path_realgeography = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/geography/'

path = path_aquaplanet

### Data Generator using RH

In [5]:
#scale_dict_RH = load_pickle('/home/ankitesh/CBrain_project/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling_2.pkl')
scale_dict_RH = scale_dict.copy()
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%

in_vars_RH = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','QRL','QRS']

# New GP path below
TRAINFILE_RH = '2021_01_24_O3_small_shuffle.nc'
NORMFILE_RH = '2021_02_01_NORM_O3_RH_small.nc'
    
# Comet/Ankitesh path below
# TRAINFILE_RH = 'CI_RH_M4K_NORM_train_shuffle.nc'
# NORMFILE_RH = 'CI_RH_M4K_NORM_norm.nc'
# VALIDFILE_RH = 'CI_RH_M4K_NORM_valid.nc'

In [6]:
train_gen_RH = DataGenerator(
    data_fn = path+TRAINFILE_RH,
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = path+NORMFILE_RH,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict_RH,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using QSATdeficit

We only need the norm file for this generator as we are solely using it as an input to determine the right normalization for the combined generator

In [7]:
# New GP path below
TRAINFILE_QSATdeficit = '2021_02_01_O3_QSATdeficit_small_shuffle.nc'
NORMFILE_QSATdeficit = '2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [8]:
in_vars_QSATdeficit = ['QSATdeficit','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_QSATdeficit = ['PHQ','TPHYSTND','QRL','QRS']

In [9]:
train_gen_QSATdeficit = DataGenerator(
    data_fn = path+TRAINFILE_QSATdeficit,
    input_vars = in_vars_QSATdeficit,
    output_vars = out_vars_QSATdeficit,
    norm_fn = path+NORMFILE_QSATdeficit,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using TNS

In [10]:
in_vars = ['QBP','TfromNS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_TNS = '2021_02_01_O3_TfromNS_small_shuffle.nc'
NORMFILE_TNS = '2021_02_01_NORM_O3_TfromNS_small.nc'
VALIDFILE_TNS = 'CI_TNS_M4K_NORM_valid.nc'

In [11]:
train_gen_TNS = DataGenerator(
    data_fn = path+TRAINFILE_TNS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_TNS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using BCONS

In [12]:
in_vars = ['QBP','BCONS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_BCONS = '2021_02_01_O3_BCONS_small_shuffle.nc'
NORMFILE_BCONS = '2021_02_01_NORM_O3_BCONS_small.nc'

In [13]:
train_gen_BCONS = DataGenerator(
    data_fn = path+TRAINFILE_BCONS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_BCONS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using NSto220

In [14]:
in_vars = ['QBP','T_NSto220','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_T_NSto220 = '2021_03_31_O3_T_NSto220_small.nc'
NORMFILE_T_NSto220 = '2021_03_31_NORM_O3_T_NSto220_small.nc'

In [15]:
train_gen_T_NSto220 = DataGenerator(
    data_fn = path+TRAINFILE_T_NSto220,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_T_NSto220,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsDELQ

In [16]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsDELQ = '2021_02_01_O3_LHF_nsDELQ_small_shuffle.nc'
NORMFILE_LHF_nsDELQ = '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

In [17]:
train_gen_LHF_nsDELQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsDELQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsDELQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsQ

In [18]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsQ = '2021_02_01_O3_LHF_nsQ_small_shuffle.nc'
NORMFILE_LHF_nsQ = '2021_02_01_NORM_O3_LHF_nsQ_small.nc'

In [19]:
train_gen_LHF_nsQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator Combined (latest flexible version)

In [20]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
#if path==path_aquaplanet: out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']
out_vars = ['PHQ','TPHYSTND','QRL','QRS']

In [21]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In physical space
TRAINFILE = '2021_03_18_O3_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_03_18_O3_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_18_RG_TRAIN_M4K_shuffle.nc'

# In percentile space
#TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
#TRAINFILE = '2021_01_24_O3_small_shuffle.nc'
#VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
#TESTFILE = '2021_04_09_PERC_TEST_P4K.nc'

Old data generator by Ankitesh

Improved flexible data generator

In [22]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [23]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [24]:
test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [25]:
test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

## Add callback class to track loss on multiple sets during training

From [https://stackoverflow.com/questions/47731935/using-multiple-validation-sets-with-keras]

In [26]:
test_diffgeog_gen_CI[0][0].shape

(8192, 64)

In [26]:
np.argwhere(np.isnan(test_gen_CI[0][1]))

array([], shape=(0, 2), dtype=int64)

In [27]:
np.argwhere(np.isnan(test_gen_CI[0][0]))

array([], shape=(0, 2), dtype=int64)

In [27]:
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            valid_generator,valid_name = validation_set
            #tf.print('Results')
            results = self.model.evaluate_generator(generator=valid_generator)
            #tf.print(results)

            for metric, result in zip(self.model.metrics_names,[results]):
                #tf.print(metric,result)
                valuename = valid_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

## Quick test to develop custom loss fx (no loss tracking across multiple datasets)

#### Input and Output Rescaling (T=BCONS)

In [28]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [29]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [30]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
_________________________________________________________________
tf_op_layer_Sigmoid (TensorF [(None, 120)]             0         
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [33]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

In [35]:
pdf = {}

In [36]:
for ipath,path in enumerate([TRAINFILE,VALIDFILE,TESTFILE_DIFFCLIMATE,TESTFILE_DIFFGEOG]):
    hf = open(pathPKL+'/'+path+'_PERC.pkl','rb')
    pdf[path] = pickle.load(hf)

In [55]:
def mse_physical(pdf):
    
    def loss(y_true,y_pred):
        y_true_physical = tf.identity(y_true)
        y_pred_physical = tf.identity(y_pred)
        for ilev in range(120):
            y_true_physical[:,ilev] = \
            tfp.math.interp_regular_1d_grid(y_true[:,ilev],
                                            x_ref_min=0,x_ref_max=1,y_ref=pdf[:,ilev])
            y_pred_physical[:,ilev] = \
            tfp.math.interp_regular_1d_grid(y_pred[:,ilev],
                                            x_ref_min=0,x_ref_max=1,y_ref=pdf[:,ilev])
        return tf.mean(tf.math.squared_difference(y_pred, y_true), axis=-1)
    
    return loss 

In [56]:
# model = load_model('model.h5', 
#                    custom_objects={'loss': asymmetric_loss(alpha)})

In [57]:
model.compile(tf.keras.optimizers.Adam(), 
              loss=mse_physical(pdf=np.float32(pdf['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['PERC_array'][:,94:])))

TypeError: 'Tensor' object does not support item assignment

In [38]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
#save_name = '2021_06_12_LOGI_PERC_RH_BCONS_LHF_nsDELQ'
save_name = '2021_06_12_Test'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [39]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [40]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
1577/5759 [=======>......................] - ETA: 27:04 - loss: 0.0337

KeyboardInterrupt: 

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Models tracking losses across climates and geography (Based on cold Aquaplanet)

### MLR or Logistic regression

#### BF

In [76]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [77]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [79]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR'

In [80]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [81]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [82]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5758/5759 [============================>.] - ETA: 0s - loss: 341.8513

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=T-TNS)

In [28]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [29]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [32]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR_RH_TfromNS_LHF_nsDELQ'

In [33]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [34]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 3237s 562ms/step - loss: 333.7198 - val_loss: 318.1690
Epoch 2/20
5759/5759 [==============================] - 3207s 557ms/step - loss: 310.0145 - val_loss: 306.5839
Epoch 3/20
5759/5759 [==============================] - 3198s 555ms/step - loss: 302.6933 - val_loss: 301.9040
Epoch 4/20
5758/5759 [============================>.] - ETA: 0s - loss: 299.3518

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=T-TNS)

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In percentile space
TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_04_09_PERC_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_24_RG_PERC_TRAIN_M4K_shuffle.nc'

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_LOGI_PERC_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_LOGI_PERC_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### NN

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
#if path==path_aquaplanet: out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']
out_vars = ['PHQ','TPHYSTND','QRL','QRS']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In physical space
TRAINFILE = '2021_03_18_O3_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_03_18_O3_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_18_RG_TRAIN_M4K_shuffle.nc'

In [ ]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [ ]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [ ]:
test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')



#### BF

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=T-TNS)

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=T-TNS)

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In percentile space
TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_04_09_PERC_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_24_RG_PERC_TRAIN_M4K_shuffle.nc'

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_PERC_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_PERC_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Models tracking losses across climates/geography (Warm to Cold)

## Brute-Force Model

### Climate-invariant (T,Q,PS,S0,SHF,LHF)->($\dot{T}$,$\dot{q}$,RADFLUX)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(64, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_temp'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

### Ozone (T,Q,$O_{3}$,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(94,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_O3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

### No Ozone (T,Q,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_noO3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
# Nep = 15
# model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
#               callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
              callbacks=[earlyStopping, mcp_save_pos])

### BF linear version

In [36]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [39]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_MLR_PERC'

In [40]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [41]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [42]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/15
5759/5759 [==============================] - 1184s 206ms/step - loss: 0.0510 - val_loss: 0.0429
Epoch 2/15
5759/5759 [==============================] - 1035s 180ms/step - loss: 0.0419 - val_loss: 0.0427
Epoch 3/15
5759/5759 [==============================] - 753s 131ms/step - loss: 0.0417 - val_loss: 0.0425
Epoch 4/15
5759/5759 [==============================] - 726s 126ms/step - loss: 0.0416 - val_loss: 0.0424
Epoch 5/15
5759/5759 [==============================] - 765s 133ms/step - loss: 0.0415 - val_loss: 0.0424
Epoch 6/15
5759/5759 [==============================] - 713s 124ms/step - loss: 0.0415 - val_loss: 0.0424
Epoch 7/15
5759/5759 [==============================] - 756s 131ms/step - loss: 0.0415 - val_loss: 0.0423
Epoch 8/15
5759/5759 [==============================] - 766s 133ms/step - loss: 0.0415 - val_loss: 0.0423
Epoch 9/15
5759/5759 [==============================] - 763s 132ms/step - loss: 0.0415 - val_loss: 0.0423
Epoch 10/15
5759/5759 [=====================

In [43]:
history.history

{'loss': [0.050982531596102595,
  0.04185105333621251,
  0.04165498730630319,
  0.04157832884143202,
  0.04153471013858079,
  0.041505846159183686,
  0.04148416520251771,
  0.04146769321144383,
  0.04145484425892099,
  0.04144392009125254,
  0.04143447252735948,
  0.04142640940075244,
  0.04141932996618994,
  0.04141429779437562,
  0.04140815107314552],
 'val_loss': [0.042909197750994055,
  0.04265815539936504,
  0.04253125871611655,
  0.0423782235956507,
  0.04237728513551328,
  0.04235305504845943,
  0.04230615223943935,
  0.04227608388790575,
  0.0423463198359328,
  0.042378264729721796,
  0.042240786968061286,
  0.04227235703251836,
  0.042130285002797335,
  0.04226516442969454,
  0.04249188080922277],
 'testP4K_loss': [0.04237089483123066,
  0.04204679281155743,
  0.041950938805466936,
  0.041866756795589787,
  0.04182252665865533,
  0.04181972140371545,
  0.041768535935374836,
  0.041756746478308786,
  0.041807406456754515,
  0.04179418787080238,
  0.041728623833007836,
  0.04172

In [44]:
hist_rec = history.history

In [45]:
hist_rec

{'loss': [0.050982531596102595,
  0.04185105333621251,
  0.04165498730630319,
  0.04157832884143202,
  0.04153471013858079,
  0.041505846159183686,
  0.04148416520251771,
  0.04146769321144383,
  0.04145484425892099,
  0.04144392009125254,
  0.04143447252735948,
  0.04142640940075244,
  0.04141932996618994,
  0.04141429779437562,
  0.04140815107314552],
 'val_loss': [0.042909197750994055,
  0.04265815539936504,
  0.04253125871611655,
  0.0423782235956507,
  0.04237728513551328,
  0.04235305504845943,
  0.04230615223943935,
  0.04227608388790575,
  0.0423463198359328,
  0.042378264729721796,
  0.042240786968061286,
  0.04227235703251836,
  0.042130285002797335,
  0.04226516442969454,
  0.04249188080922277],
 'testP4K_loss': [0.04237089483123066,
  0.04204679281155743,
  0.041950938805466936,
  0.041866756795589787,
  0.04182252665865533,
  0.04181972140371545,
  0.041768535935374836,
  0.041756746478308786,
  0.041807406456754515,
  0.04179418787080238,
  0.041728623833007836,
  0.04172

In [46]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BF Logistic version

In [69]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [70]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               7800      
_________________________________________________________________
tf_op_layer_Sigmoid_1 (Tenso [(None, 120)]             0         
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [71]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [72]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_Log_PERC'

In [73]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [74]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [75]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/15
5759/5759 [==============================] - 794s 138ms/step - loss: 0.0492 - val_loss: 0.0427
Epoch 2/15
5759/5759 [==============================] - 761s 132ms/step - loss: 0.0411 - val_loss: 0.0412
Epoch 3/15
5759/5759 [==============================] - 842s 146ms/step - loss: 0.0405 - val_loss: 0.0410
Epoch 4/15
5759/5759 [==============================] - 691s 120ms/step - loss: 0.0403 - val_loss: 0.0408
Epoch 5/15
5759/5759 [==============================] - 727s 126ms/step - loss: 0.0402 - val_loss: 0.0408
Epoch 6/15
5759/5759 [==============================] - 727s 126ms/step - loss: 0.0401 - val_loss: 0.0407
Epoch 7/15
5759/5759 [==============================] - 738s 128ms/step - loss: 0.0401 - val_loss: 0.0406
Epoch 8/15
5759/5759 [==============================] - 740s 129ms/step - loss: 0.0400 - val_loss: 0.0406
Epoch 9/15
5759/5759 [==============================] - 769s 133ms/step - loss: 0.0400 - val_loss: 0.0406
Epoch 10/15
5759/5759 [=======================

In [76]:
history.history

{'loss': [0.04923291314722536,
  0.04110610745569074,
  0.04050010521521811,
  0.040299923023502326,
  0.040188955621450057,
  0.040115493637423776,
  0.040061687980442955,
  0.04002036269023862,
  0.039987151799029744,
  0.03995977671317889,
  0.03993697982862977,
  0.03991729508678539,
  0.0399002306710558,
  0.03988536803773406,
  0.03987165327315054],
 'val_loss': [0.04265024852781968,
  0.041217454327464445,
  0.04097958452281418,
  0.04079991503641219,
  0.04078920084213819,
  0.04070766954445322,
  0.04064866158667609,
  0.0406280841281429,
  0.04056495156529776,
  0.04057289374916206,
  0.040560972740690364,
  0.0405654035945343,
  0.0404891143274493,
  0.040506530972142435,
  0.0404415049514598],
 'testP4K_loss': [0.07411513028238494,
  0.07390664614972449,
  0.07520650478889554,
  0.07573994763074626,
  0.0764975735435722,
  0.0768441669025755,
  0.07727208656825946,
  0.07749857328113179,
  0.07786896164909984,
  0.07816397840018766,
  0.07837125465184781,
  0.07873877331958

In [77]:
hist_rec = history.history

In [78]:
hist_rec

{'loss': [0.04923291314722536,
  0.04110610745569074,
  0.04050010521521811,
  0.040299923023502326,
  0.040188955621450057,
  0.040115493637423776,
  0.040061687980442955,
  0.04002036269023862,
  0.039987151799029744,
  0.03995977671317889,
  0.03993697982862977,
  0.03991729508678539,
  0.0399002306710558,
  0.03988536803773406,
  0.03987165327315054],
 'val_loss': [0.04265024852781968,
  0.041217454327464445,
  0.04097958452281418,
  0.04079991503641219,
  0.04078920084213819,
  0.04070766954445322,
  0.04064866158667609,
  0.0406280841281429,
  0.04056495156529776,
  0.04057289374916206,
  0.040560972740690364,
  0.0405654035945343,
  0.0404891143274493,
  0.040506530972142435,
  0.0404415049514598],
 'testP4K_loss': [0.07411513028238494,
  0.07390664614972449,
  0.07520650478889554,
  0.07573994763074626,
  0.0764975735435722,
  0.0768441669025755,
  0.07727208656825946,
  0.07749857328113179,
  0.07786896164909984,
  0.07816397840018766,
  0.07837125465184781,
  0.07873877331958

In [79]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BF NN version with test loss tracking

In [44]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [45]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [47]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_08_NN6L'

In [48]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [49]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [50]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 2542s 441ms/step - loss: 342.0934 - val_loss: 329.9304
Epoch 2/20
5759/5759 [==============================] - 2701s 469ms/step - loss: 320.9061 - val_loss: 316.9567
Epoch 3/20
5759/5759 [==============================] - 2452s 426ms/step - loss: 311.2353 - val_loss: 309.6577
Epoch 4/20
5759/5759 [==============================] - 2254s 391ms/step - loss: 305.5034 - val_loss: 304.9600
Epoch 5/20
5759/5759 [==============================] - 1731s 301ms/step - loss: 301.7379 - val_loss: 301.6836
Epoch 6/20
5759/5759 [==============================] - 1117s 194ms/step - loss: 299.0327 - val_loss: 299.1967
Epoch 7/20
5759/5759 [==============================] - 816s 142ms/step - loss: 296.9427 - val_loss: 297.2431
Epoch 8/20
5759/5759 [==============================] - 713s 124ms/step - loss: 295.2764 - val_loss: 295.7120
Epoch 9/20
5759/5759 [==============================] - 694s 121ms/step - loss: 293.9242 - val_loss: 294.4352
Epoc

In [51]:
history.history

{'loss': [342.09338355590165,
  320.9061308653948,
  311.23525867438974,
  305.5033564990667,
  301.7378943407364,
  299.03272853665186,
  296.9426967053182,
  295.27639429510043,
  293.9242286688753,
  292.8105507327525,
  291.88271119160294,
  291.10301101189106,
  290.43950267433394,
  289.868749243287,
  289.3729577492583,
  288.93778015867997,
  288.5534648097085,
  288.2098106308765,
  287.90122331664634,
  287.6208748516885],
 'val_loss': [329.9303815202876,
  316.9566871866662,
  309.6576747567941,
  304.96000983386693,
  301.6835738202586,
  299.1966588354587,
  297.243100628537,
  295.7119940864864,
  294.43520978061343,
  293.3954472839883,
  292.5363609783128,
  291.8279773312142,
  291.20056871985776,
  290.6817859751116,
  290.2149793161854,
  289.82551079177665,
  289.4662798184087,
  289.1482212480884,
  288.8622394983756,
  288.5948049347219],
 'testP4K_loss': [755.1666537160246,
  749.3020249286134,
  764.911121189238,
  790.1407622900099,
  815.8143627241674,
  838.4

In [52]:
hist_rec = history.history

In [53]:
hist_rec

{'loss': [342.09338355590165,
  320.9061308653948,
  311.23525867438974,
  305.5033564990667,
  301.7378943407364,
  299.03272853665186,
  296.9426967053182,
  295.27639429510043,
  293.9242286688753,
  292.8105507327525,
  291.88271119160294,
  291.10301101189106,
  290.43950267433394,
  289.868749243287,
  289.3729577492583,
  288.93778015867997,
  288.5534648097085,
  288.2098106308765,
  287.90122331664634,
  287.6208748516885],
 'val_loss': [329.9303815202876,
  316.9566871866662,
  309.6576747567941,
  304.96000983386693,
  301.6835738202586,
  299.1966588354587,
  297.243100628537,
  295.7119940864864,
  294.43520978061343,
  293.3954472839883,
  292.5363609783128,
  291.8279773312142,
  291.20056871985776,
  290.6817859751116,
  290.2149793161854,
  289.82551079177665,
  289.4662798184087,
  289.1482212480884,
  288.8622394983756,
  288.5948049347219],
 'testP4K_loss': [755.1666537160246,
  749.3020249286134,
  764.911121189238,
  790.1407622900099,
  815.8143627241674,
  838.4

In [54]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH Logistic version

In [86]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [87]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               7800      
_________________________________________________________________
tf_op_layer_Sigmoid_2 (Tenso [(None, 120)]             0         
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [89]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_Log_PERC_RH'

In [90]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [91]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [92]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/15
5759/5759 [==============================] - 1103s 191ms/step - loss: 0.0467 - val_loss: 0.0405
Epoch 2/15
5759/5759 [==============================] - 1136s 197ms/step - loss: 0.0392 - val_loss: 0.0394
Epoch 3/15
5759/5759 [==============================] - 1123s 195ms/step - loss: 0.0389 - val_loss: 0.0394
Epoch 4/15
5759/5759 [==============================] - 1115s 194ms/step - loss: 0.0388 - val_loss: 0.0393
Epoch 5/15
5759/5759 [==============================] - 1128s 196ms/step - loss: 0.0387 - val_loss: 0.0393
Epoch 6/15
5759/5759 [==============================] - 1082s 188ms/step - loss: 0.0387 - val_loss: 0.0393
Epoch 7/15
5759/5759 [==============================] - 1136s 197ms/step - loss: 0.0387 - val_loss: 0.0393
Epoch 8/15
5759/5759 [==============================] - 1069s 186ms/step - loss: 0.0387 - val_loss: 0.0392
Epoch 9/15
5759/5759 [==============================] - 1093s 190ms/step - loss: 0.0387 - val_loss: 0.0392
Epoch 10/15
5759/5759 [==============

In [93]:
history.history

{'loss': [0.0467287344276843,
  0.0392072251961502,
  0.038853517252732575,
  0.03877987418298528,
  0.03874506841666439,
  0.03872233596301302,
  0.03870567677141188,
  0.03869225124110984,
  0.03868109595084236,
  0.03867188213775712,
  0.0386642090824872,
  0.03865747418455789,
  0.03865132474455802,
  0.03864620624032156,
  0.03864140413024914],
 'val_loss': [0.040451113186371485,
  0.03944033429256318,
  0.03935017663365135,
  0.03931213896163092,
  0.03929163821896072,
  0.03926565892478177,
  0.039275243295873806,
  0.039249983558271286,
  0.0392355400208758,
  0.03925549958671793,
  0.03922620319762756,
  0.03919682341832721,
  0.03921325347639839,
  0.039212426125584225,
  0.03919467062651657],
 'testP4K_loss': [0.06156855558834964,
  0.05892425673790308,
  0.05871588077863804,
  0.058408490616755036,
  0.05826237939559019,
  0.05821150895383091,
  0.058247450274676975,
  0.05818161563383118,
  0.05822653928238749,
  0.05826851157272707,
  0.05817507403042966,
  0.058177342630

In [94]:
hist_rec = history.history

In [95]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH linear version

In [32]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [35]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH'

In [36]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [37]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [38]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
   3/5759 [..............................] - ETA: 56:44 - loss: 362.5890  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### QSATdeficit linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_QSATdeficit'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### TfromNS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_TfromNS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BCONS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_BCONS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Mixed Model

In [24]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [27]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_BCONS'

In [28]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [29]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [30]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 8098s 1s/step - loss: 324.9880 - val_loss: 313.2678
Epoch 2/10
5759/5759 [==============================] - 12555s 2s/step - loss: 308.3165 - val_loss: 307.0678
Epoch 3/10
5759/5759 [==============================] - 13642s 2s/step - loss: 304.0913 - val_loss: 304.2872
Epoch 4/10
5759/5759 [==============================] - 10465s 2s/step - loss: 301.8938 - val_loss: 302.7330
Epoch 5/10
5759/5759 [==============================] - 7909s 1s/step - loss: 300.5873 - val_loss: 301.7510
Epoch 6/10
5759/5759 [==============================] - 8526s 1s/step - loss: 299.7002 - val_loss: 301.0397
Epoch 7/10
5759/5759 [==============================] - 7230s 1s/step - loss: 299.0066 - val_loss: 300.3937
Epoch 8/10
5759/5759 [==============================] - 8294s 1s/step - loss: 298.4328 - val_loss: 299.8434
Epoch 9/10
5759/5759 [==============================] - 8250s 1s/step - loss: 297.9426 - val_loss: 299.4925
Epoch 10/10
5759/5759 [==

In [31]:
hist_rec = history.history

In [32]:
hist_rec

{'loss': [324.98798001675704,
  308.31648811596017,
  304.0912586973243,
  301.8937511361293,
  300.5873382457078,
  299.7002091802725,
  299.0065637804771,
  298.4328209674489,
  297.9425933249689,
  297.5149222372472],
 'val_loss': [313.2678370875624,
  307.0677993880396,
  304.2871500202989,
  302.73299462630064,
  301.75103527639715,
  301.03966337007205,
  300.39373488234764,
  299.8433922314147,
  299.4924658236212,
  299.0508612894959],
 'testP4K_loss': [696.3224985280676,
  686.2635380876054,
  682.7061903721445,
  680.7840038270242,
  679.5804400619129,
  678.422916433854,
  677.2276842855165,
  676.2437589546028,
  675.5998048438668,
  674.6606088692057]}

In [33]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+(T-TNS)

### RH+NSto220

In [39]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [40]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [42]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_31_MLR_RH_NSto220'

In [43]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [44]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [45]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 10546s 2s/step - loss: 333.2154 - val_loss: 318.1506
Epoch 2/10
5759/5759 [==============================] - 10194s 2s/step - loss: 310.9067 - val_loss: 308.3093
Epoch 3/10
5759/5759 [==============================] - 9862s 2s/step - loss: 304.7300 - val_loss: 304.2289
Epoch 4/10
5759/5759 [==============================] - 11121s 2s/step - loss: 301.8291 - val_loss: 302.1135
Epoch 5/10
5759/5759 [==============================] - 10748s 2s/step - loss: 300.2339 - val_loss: 300.8775
Epoch 6/10
5759/5759 [==============================] - 10117s 2s/step - loss: 299.1923 - val_loss: 299.9938
Epoch 7/10
5759/5759 [==============================] - 10174s 2s/step - loss: 298.3948 - val_loss: 299.3052
Epoch 8/10
5759/5759 [==============================] - 9887s 2s/step - loss: 297.7428 - val_loss: 298.7333
Epoch 9/10
5759/5759 [==============================] - 10299s 2s/step - loss: 297.1934 - val_loss: 298.2455
Epoch 10/10
5759/5759

In [46]:
hist_rec = history.history

In [47]:
hist_rec

{'loss': [333.21542532209264,
  310.90667960419796,
  304.72998016670067,
  301.8290961309116,
  300.2339088743316,
  299.19231390816446,
  298.39477655642935,
  297.74276377828113,
  297.19338681379486,
  296.7217506637878],
 'val_loss': [318.15060213358146,
  308.30926291885646,
  304.22893252623965,
  302.1134895894682,
  300.8775425180225,
  299.9937965616662,
  299.30517368746393,
  298.73325744039545,
  298.2454878341621,
  297.8241368286815],
 'testP4K_loss': [723.0185877851295,
  698.1272752300885,
  685.258197891926,
  679.2784368682783,
  676.55788616959,
  674.8558076801512,
  673.4969380158018,
  672.4355979235329,
  671.423601997825,
  670.7109736936912]}

In [48]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+LHF_nsQ

In [39]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [40]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [42]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_LHF_nsQ'

In [43]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [44]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [45]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 7449s 1s/step - loss: 332.8352 - val_loss: 318.1170
Epoch 2/10
5759/5759 [==============================] - 5519s 958ms/step - loss: 311.1496 - val_loss: 308.7574
Epoch 3/10
5759/5759 [==============================] - 5822s 1s/step - loss: 305.3264 - val_loss: 304.9336
Epoch 4/10
5759/5759 [==============================] - 6582s 1s/step - loss: 302.6262 - val_loss: 302.9958
Epoch 5/10
5759/5759 [==============================] - 7103s 1s/step - loss: 301.1792 - val_loss: 301.9337
Epoch 6/10
5759/5759 [==============================] - 14946s 3s/step - loss: 300.2849 - val_loss: 301.1922
Epoch 7/10
5759/5759 [==============================] - 4869s 846ms/step - loss: 299.6383 - val_loss: 300.6844
Epoch 8/10
5759/5759 [==============================] - 5781s 1s/step - loss: 299.1321 - val_loss: 300.2569
Epoch 9/10
5759/5759 [==============================] - 6748s 1s/step - loss: 298.7180 - val_loss: 299.8978
Epoch 10/10
5759/5759

In [46]:
hist_rec = history.history

In [47]:
hist_rec

{'loss': [332.83523499690364,
  311.1495506735721,
  305.32637336838263,
  302.6261628686316,
  301.1792370861946,
  300.28492045017515,
  299.6382695135298,
  299.1321185072753,
  298.7179960372403,
  298.3687616935474],
 'val_loss': [318.11703873779254,
  308.75738331521546,
  304.9335873032069,
  302.9957635315034,
  301.9337132302216,
  301.1921791930977,
  300.68438835990605,
  300.2568701773645,
  299.89779036501074,
  299.60179900476817],
 'testP4K_loss': [732.1974313007879,
  706.4723622909272,
  691.7300854846619,
  684.7900059335971,
  681.8280336922402,
  679.9574934180021,
  678.7282812103871,
  677.7188479585591,
  676.8431889352383,
  676.1222872974321]}

In [48]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsDELQ NN version with test loss tracking

In [68]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [69]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 128)               0   

In [70]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [71]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_09_NN7L_RH_TfromNS_LHF_nsDELQ'

In [72]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [73]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 2906s 505ms/step - loss: 202.5537 - val_loss: 189.6939
Epoch 2/20
5759/5759 [==============================] - 2024s 352ms/step - loss: 185.1673 - val_loss: 182.6220
Epoch 3/20
5759/5759 [==============================] - 1676s 291ms/step - loss: 180.0439 - val_loss: 180.6006
Epoch 4/20
5759/5759 [==============================] - 1666s 289ms/step - loss: 177.2701 - val_loss: 176.7251
Epoch 5/20
5759/5759 [==============================] - 1665s 289ms/step - loss: 175.5439 - val_loss: 175.3660
Epoch 6/20
5759/5759 [==============================] - 1634s 284ms/step - loss: 174.3043 - val_loss: 174.8005
Epoch 7/20
5759/5759 [==============================] - 1668s 290ms/step - loss: 173.3022 - val_loss: 173.3208
Epoch 8/20
5759/5759 [==============================] - 1678s 291ms/step - loss: 172.4542 - val_loss: 172.6152
Epoch 9/20
5759/5759 [==============================] - 2503s 435ms/step - loss: 171.7443 - val_loss: 172.0109
E

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsQ

In [68]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [69]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [72]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_TfromNS_LHF_nsQ'

In [73]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [74]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [75]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 6478s 1s/step - loss: 334.0426 - val_loss: 318.8078
Epoch 2/10
5759/5759 [==============================] - 7037s 1s/step - loss: 310.9031 - val_loss: 307.7163
Epoch 3/10
5759/5759 [==============================] - 5910s 1s/step - loss: 304.0670 - val_loss: 303.4669
Epoch 4/10
5759/5759 [==============================] - 7744s 1s/step - loss: 301.0845 - val_loss: 301.4636
Epoch 5/10
5759/5759 [==============================] - 7767s 1s/step - loss: 299.5355 - val_loss: 300.3262
Epoch 6/10
5759/5759 [==============================] - 7525s 1s/step - loss: 298.5930 - val_loss: 299.6363
Epoch 7/10
5759/5759 [==============================] - 7967s 1s/step - loss: 297.9281 - val_loss: 299.1007
Epoch 8/10
5759/5759 [==============================] - 6677s 1s/step - loss: 297.4247 - val_loss: 298.6999
Epoch 9/10
5759/5759 [==============================] - 6912s 1s/step - loss: 297.0260 - val_loss: 298.3802
Epoch 10/10
5759/5759 [=====

In [76]:
hist_rec = history.history

In [77]:
hist_rec

{'loss': [334.04261498667336,
  310.90313761506275,
  304.06700017431956,
  301.08452700392866,
  299.5355056150648,
  298.59301007193477,
  297.92813114855807,
  297.42471469800324,
  297.02595976008,
  296.69932817398495],
 'val_loss': [318.8077631269736,
  307.7162972562862,
  303.4669281734807,
  301.4636359146121,
  300.3261635153653,
  299.6362964928039,
  299.10066872735194,
  298.6998974468755,
  298.3802061189176,
  298.1298349087428],
 'testP4K_loss': [717.7692014099695,
  692.5093093676612,
  681.1892812258153,
  677.2928423535407,
  675.2491797104337,
  673.9023133940009,
  672.9248305401367,
  672.2034651915096,
  671.5856063464887,
  671.1150249686433]}

In [78]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+BCONS+LHF_nsDELQ

In [95]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [96]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [98]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_BCONS_LHF_nsDELQ'

In [99]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [100]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [101]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 8133s 1s/step - loss: 330.7609 - val_loss: 316.7712
Epoch 2/10
5759/5759 [==============================] - 6660s 1s/step - loss: 310.4906 - val_loss: 307.8904
Epoch 3/10
5759/5759 [==============================] - 7116s 1s/step - loss: 304.7100 - val_loss: 303.9924
Epoch 4/10
5759/5759 [==============================] - 7062s 1s/step - loss: 301.7960 - val_loss: 301.8254
Epoch 5/10
5759/5759 [==============================] - 8137s 1s/step - loss: 300.0964 - val_loss: 300.5073
Epoch 6/10
5759/5759 [==============================] - 7596s 1s/step - loss: 298.9904 - val_loss: 299.6046
Epoch 7/10
5759/5759 [==============================] - 7545s 1s/step - loss: 298.1876 - val_loss: 298.9532
Epoch 8/10
5759/5759 [==============================] - 7470s 1s/step - loss: 297.5743 - val_loss: 298.4452
Epoch 9/10
5759/5759 [==============================] - 7456s 1s/step - loss: 297.0949 - val_loss: 298.0184
Epoch 10/10
5759/5759 [=====

In [102]:
hist_rec = history.history

In [103]:
hist_rec

{'loss': [330.76085305019353,
  310.4906432967692,
  304.7099609745938,
  301.7960417360497,
  300.09642175858914,
  298.9904117309337,
  298.18756228044725,
  297.57428517435176,
  297.0949075244785,
  296.709771785746],
 'val_loss': [316.77116454788614,
  307.8904337055013,
  303.99240362525046,
  301.8254467632818,
  300.5073347083597,
  299.6046408204614,
  298.95319436894175,
  298.4452129081611,
  298.01840061586466,
  297.69113439902117],
 'testP4K_loss': [702.3246120228104,
  686.3533080012243,
  680.0862056377094,
  676.7398991455286,
  674.6481522557676,
  673.1816021796275,
  672.0055764137222,
  671.111931773641,
  670.3688568089174,
  669.7110506886809]}

In [104]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsDELQ

In [55]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [56]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [58]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_01_MLR_RH_NSto220_LHF_nsDELQ'

In [59]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [60]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [61]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 11407s 2s/step - loss: 333.0526 - val_loss: 317.6788
Epoch 2/10
5759/5759 [==============================] - 10810s 2s/step - loss: 310.2102 - val_loss: 307.3595
Epoch 3/10
5759/5759 [==============================] - 10216s 2s/step - loss: 303.6873 - val_loss: 302.9961
Epoch 4/10
5759/5759 [==============================] - 10683s 2s/step - loss: 300.5542 - val_loss: 300.6824
Epoch 5/10
5759/5759 [==============================] - 10550s 2s/step - loss: 298.8112 - val_loss: 299.3256
Epoch 6/10
5759/5759 [==============================] - 10292s 2s/step - loss: 297.7034 - val_loss: 298.4254
Epoch 7/10
5759/5759 [==============================] - 10560s 2s/step - loss: 296.9044 - val_loss: 297.7670
Epoch 8/10
5759/5759 [==============================] - 11866s 2s/step - loss: 296.3028 - val_loss: 297.2482
Epoch 9/10
5759/5759 [==============================] - 10787s 2s/step - loss: 295.8395 - val_loss: 296.8615
Epoch 10/10
5759/57

In [62]:
hist_rec = history.history

In [63]:
hist_rec

{'loss': [333.05256303355685,
  310.2102426647167,
  303.6872879666863,
  300.55415829901773,
  298.81120031363105,
  297.7034077711415,
  296.90441165903536,
  296.30281556532356,
  295.83954845970965,
  295.4770164907051],
 'val_loss': [317.6788116925534,
  307.3594649500833,
  302.9960944944573,
  300.68239808442047,
  299.32556045328016,
  298.42535546180176,
  297.7669507749966,
  297.24817319492274,
  296.86145162218895,
  296.5717960327293],
 'testP4K_loss': [721.7310292205681,
  696.0703459624854,
  682.7236189375567,
  676.256974551415,
  672.9861661186186,
  670.9493864272823,
  669.4111307171855,
  668.1374724690269,
  667.1579007533959,
  666.4447404165944]}

In [64]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsQ

In [77]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [78]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [80]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_03_MLR_RH_NSto220_LHF_nsQ'

In [81]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [82]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [83]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 9714s 2s/step - loss: 333.0516 - val_loss: 317.9425
Epoch 2/10
5759/5759 [==============================] - 6439s 1s/step - loss: 310.6932 - val_loss: 308.0465
Epoch 3/10
5759/5759 [==============================] - 5183s 900ms/step - loss: 304.5171 - val_loss: 304.0613
Epoch 4/10
5759/5759 [==============================] - 6201s 1s/step - loss: 301.7091 - val_loss: 302.0904
Epoch 5/10
5759/5759 [==============================] - 6448s 1s/step - loss: 300.2337 - val_loss: 301.0068
Epoch 6/10
5759/5759 [==============================] - 6364s 1s/step - loss: 299.3400 - val_loss: 300.3276
Epoch 7/10
5759/5759 [==============================] - 5246s 911ms/step - loss: 298.7033 - val_loss: 299.8050
Epoch 8/10
5759/5759 [==============================] - 5084s 883ms/step - loss: 298.2143 - val_loss: 299.3951
Epoch 9/10
5759/5759 [==============================] - 7377s 1s/step - loss: 297.8223 - val_loss: 299.0667
Epoch 10/10
5759/57

In [84]:
hist_rec = history.history

In [85]:
hist_rec

{'loss': [333.05156581512364,
  310.6931948111193,
  304.51710277887906,
  301.70909371684877,
  300.2336531922436,
  299.34000020094226,
  298.7033277413232,
  298.2142985748487,
  297.8223111576081,
  297.49855535381016],
 'val_loss': [317.9424894715131,
  308.0464616504419,
  304.0613429893301,
  302.090381744452,
  301.0067826105784,
  300.32760883262154,
  299.8049709262554,
  299.39514361780095,
  299.06670788960923,
  298.80823058409953],
 'testP4K_loss': [722.2397246819291,
  697.4373292892026,
  684.9307260671301,
  679.3653844453859,
  676.8490013480899,
  675.4289473119096,
  674.3540836741433,
  673.5583135512015,
  672.867135846849,
  672.3943842652513]}

In [86]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()